# Programming in PySpark RDD’s

## Abstracting Data with RDDs

Resilient Distributed Datasets. Collection of data distributed across the cluster. 3 important properties of RDD:

    Resilient: Ability to withstand failures
    Distributed: Spanning across multiple machines
    Datasets: Collection of partitioned data like Arrays, Tables, Tuples.
    
Common way to create RDDs is to lead data from external datasets such as files stored in HDFS or objects in Amazon S3 bucket or lines in a text file stored locally and pass it to SparkContext's textFile method. Also RDDs can also be created from existing RDDs which we will see in the next video.

A partition is a logical division of a large distributed data set. Minimum number of partitions can be created for an RDD. The number of partitons in an RDD can always be found by using the getNumPartitions method.

### RDDs from Parallelized collections

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "pyspark-shell")

RDD = sc.parallelize(["Spark", "is", "a", "framework", "for", "Big Data processing"])

print(type(RDD))

<class 'pyspark.rdd.RDD'>


### RDDs from External Datasets


In [2]:
fileRDD = sc.textFile("README.md")

print("The file type of fileRDD is", type(fileRDD))

The file type of fileRDD is <class 'pyspark.rdd.RDD'>


### Partitions in your data

In [8]:
print("Number of partitions in fileRDD is", fileRDD.getNumPartitions())

fileRDD_part = sc.textFile("README.md", minPartitions = 5)

print("Number of partitions in fileRDD_part is", fileRDD_part.getNumPartitions())

Number of partitions in fileRDD is 1
Number of partitions in fileRDD_part is 9


## Basic RDD Transformations and Actions

RDDs in PySpark supports two different types of operations. Transformations and actions. Transformations create new RDDs and actions perform computation on the RDDs. Transformations follow Lazy evaluation. Basic RDD transformations are map, filter, flatmap and union. 

map() transformation applies a function to all elements in the RDD.

filter transformation returns a new RDD with only the elements that pass the condition.

flatmap is similar to map transformation except it returns multiple values for each element in the source RDD.

Uninon transformation returns the union of one RDD with another RDD.

Actions are the opretions that applied on RDDs to return a value after running a computation. The four basic actions are collect, take, first and count. 

Collect returns complete list of elements from the RDD.

take(N) returns an array with the first N elements of the dataset.

first prints the first element of the RDD.

count return the number of elements in the RDD.


### Map and Collect

In [7]:
numbRDD = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

cubeRDD = numbRDD.map(lambda x: x**3)

numbers_all = cubeRDD.collect()
for numb in numbers_all:
    print(numb)

1
8
27
64
125
216
343
512
729
1000


### Filter and Count

In [34]:
fileRDD = sc.textFile("README.md")
fileRDD_filter = fileRDD.filter(lambda line: "Spark" in line)

print("The total number of lines with the keyword Spark is", fileRDD_filter.count())

for line in fileRDD.take(4):
    print(line)

The total number of lines with the keyword Spark is 7
[![buildstatus](https://travis-ci.org/holdenk/learning-spark-examples.svg?branch=master)](https://travis-ci.org/holdenk/learning-spark-examples)
Examples for Learning Spark
Examples for the Learning Spark book. These examples require a number of libraries and as such have long build files. We have also added a stand alone example with minimal dependencies and a small build file


## Pair RDDs in PySpark

Most of the real world datasets are generally key/value pairs. Each row is a key and maps to one or more values. PySpark provides a special data structure called pair RDDs for this kind of data. In pair RDDs, the key refers to the identifier, whereas value refers to the data. Pair RDDs can be created from a list of key-value tuple or from a regular RDD. The first step in creating pair RDDs is to get the data into key/value form. Since pair RDDs contain tuples, we need to pass functions that operate on key-value pairs. Some of the transformations of paired RDDs:
    
    reduceByKey(): Combine values with the same key
    groupByKey(): Group values with the same key
    sortByKey(): Return an RDD sorted by the same key
    join(): Join two pair RDDs based on their key 
        

### ReduceBykey and Collect

In [36]:
Rdd = sc.parallelize([(1,2),(3,4),(3,6),(4,5)])

Rdd_reduced = Rdd.reduceByKey(lambda x, y: x+y)

for num in Rdd_reduced.collect():
    print("Key {} has {} Counts".format(num[0], num[1]))

Key 1 has 2 Counts
Key 3 has 10 Counts
Key 4 has 5 Counts


### SortByKey and Collect

In [38]:
Rdd_reduced_sort = Rdd_reduced.sortByKey(ascending=False)

for num in Rdd_reduced_sort.collect():
    print("Key {} has {} Counts".format(num[0], num[1]))

Key 4 has 5 Counts
Key 3 has 10 Counts
Key 1 has 2 Counts


## Advanced RDD Actions

reduce action is used for aggregating the elements of a regular RDD. The function should be commutative and associative.

It is not advisable to run collect action on RDDs because of the huge size of the data. In these cases, write data to a distributed storage systems such as HDFS or Amazon S3. saveAsTextFile action can be used to save RDD as a text file inside a particular directory, each partition as a separate file.

With coalece method it can be saved as a single text file.

countByKey is only available on RDDs of type (Key, Value). It counts the number of elements for each key.

collectAsMap returns the key-value pairs in the RDD as a dictionary.

These actions should only be used if the resulting data is expected to be small.

### CountingBykeys

In [40]:
Rdd = sc.parallelize([(1, 2), (3, 4), (3, 6), (4, 5)])

total = Rdd.countByKey()

print("The type of total is", type(total))

for k, v in total.items():
    print("key", k, "has", v, "counts")

The type of total is <class 'collections.defaultdict'>
key 1 has 1 counts
key 3 has 2 counts
key 4 has 1 counts


### Create a base RDD and transform it

In [43]:
baseRDD = sc.textFile("Complete_Shakespeare.txt")

splitRDD = baseRDD.flatMap(lambda x: x.split())
print("Total number of words in splitRDD:", splitRDD.count())

Total number of words in splitRDD: 128576


### Remove stop words and reduce the dataset

In [61]:
stop_words = ['i',  'me',  'my',  'myself',  'we',  'our',  'ours',  'ourselves',  'you',  'your',  'yours',  'yourself',  'yourselves',  'he',  'him',  'his',  'himself',  'she',  'her',  'hers',  'herself',  'it',  'its',  'itself',  'they',  'them',  'their',  'theirs',  'themselves',  'what',  'which',  'who',  'whom',  'this',  'that',  'these',  'those',  'am',  'is',  'are',  'was',  'were',  'be',  'been',  'being',  'have',  'has',  'had',  'having',  'do',  'does',  'did',  'doing',  'a',  'an',  'the',  'and',  'but',  'if',  'or',  'because',  'as',  'until',  'while',  'of',  'at',  'by',  'for',  'with',  'about',  'against',  'between',  'into',  'through',  'during',  'before',  'after',  'above',  'below',  'to',  'from',  'up',  'down',  'in',  'out',  'on',  'off',  'over',  'under',  'again',  'further',  'then',  'once',  'here',  'there',  'when',  'where',  'why',  'how',  'all',  'any',  'both',  'each',  'few',  'more',  'most',  'other',  'some',  'such',  'no',  'nor',  'not',  'only',  'own',  'same',  'so',  'than',  'too',  'very',  'can',  'will',  'just',  'don',  'should',  'now']

splitRDD_no_stop = splitRDD.filter(lambda x: x.lower() not in stop_words)

splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))

resultRDD = splitRDD_no_stop_words.reduceByKey(lambda x,y: x+y)

### Print word frequencies


In [70]:
for word in resultRDD.take(10):
    print(word)
    
resultRDD_swap = resultRDD.map(lambda x: (x[1], x[0]))
resultRDD_swap_sort = resultRDD_swap.sortByKey(ascending=False)
print()
for word in resultRDD_swap_sort.take(10):
    print(word)

('Project', 9)
('Gutenberg', 7)
('EBook', 1)
('Complete', 3)
('Works', 3)
('William', 11)
('Shakespeare,', 1)
('Shakespeare', 12)
('eBook', 2)
('use', 38)

(650, 'thou')
(574, 'thy')
(393, 'shall')
(311, 'would')
(295, 'good')
(286, 'thee')
(273, 'love')
(269, 'Enter')
(254, "th'")
(225, 'make')
